In [2]:
import cudf as pd
import cuml
import cupy as np
import math

In [2]:
X= list(range(60,360,3))
X=np.asarray(X)
Y=[]
for i in range(len(X)):
    Y.append(math.sin(X[i]))
Y=np.asarray(Y)

In [3]:
X

array([ 60,  63,  66,  69,  72,  75,  78,  81,  84,  87,  90,  93,  96,
        99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135,
       138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 168, 171, 174,
       177, 180, 183, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213,
       216, 219, 222, 225, 228, 231, 234, 237, 240, 243, 246, 249, 252,
       255, 258, 261, 264, 267, 270, 273, 276, 279, 282, 285, 288, 291,
       294, 297, 300, 303, 306, 309, 312, 315, 318, 321, 324, 327, 330,
       333, 336, 339, 342, 345, 348, 351, 354, 357])

In [4]:

Y

array([-0.30481062,  0.1673557 , -0.02655115, -0.11478481,  0.25382336,
       -0.38778164,  0.51397846, -0.62988799,  0.73319032, -0.82181784,
        0.89399666, -0.94828214,  0.98358775, -0.99920683,  0.99482679,
       -0.97053528,  0.92681851, -0.86455145,  0.78498039, -0.68969794,
        0.58061118, -0.45990349,  0.32999083, -0.19347339,  0.05308359,
        0.08836869, -0.22805226,  0.36317137, -0.49102159,  0.60904402,
       -0.71487643,  0.80640058, -0.88178462,  0.93951973, -0.97845035,
        0.99779728, -0.99717329,  0.97659087, -0.93646197,  0.87758979,
       -0.80115264,  0.70868041, -0.60202394,  0.48331795, -0.35493836,
        0.21945467, -0.07957859, -0.06189025,  0.20212036, -0.33830503,
        0.46771852, -0.58777062,  0.69605849, -0.79041474,  0.86895084,
       -0.93009488,  0.97262306, -0.99568419,  0.99881669, -0.98195787,
        0.94544515, -0.89000935,  0.81676   , -0.72716319,  0.62301221,
       -0.50639163,  0.37963563, -0.24528121,  0.10601749,  0.03

In [5]:
data={"X":X,"Y":Y}
data

{'X': array([ 60,  63,  66,  69,  72,  75,  78,  81,  84,  87,  90,  93,  96,
         99, 102, 105, 108, 111, 114, 117, 120, 123, 126, 129, 132, 135,
        138, 141, 144, 147, 150, 153, 156, 159, 162, 165, 168, 171, 174,
        177, 180, 183, 186, 189, 192, 195, 198, 201, 204, 207, 210, 213,
        216, 219, 222, 225, 228, 231, 234, 237, 240, 243, 246, 249, 252,
        255, 258, 261, 264, 267, 270, 273, 276, 279, 282, 285, 288, 291,
        294, 297, 300, 303, 306, 309, 312, 315, 318, 321, 324, 327, 330,
        333, 336, 339, 342, 345, 348, 351, 354, 357]),
 'Y': array([-0.30481062,  0.1673557 , -0.02655115, -0.11478481,  0.25382336,
        -0.38778164,  0.51397846, -0.62988799,  0.73319032, -0.82181784,
         0.89399666, -0.94828214,  0.98358775, -0.99920683,  0.99482679,
        -0.97053528,  0.92681851, -0.86455145,  0.78498039, -0.68969794,
         0.58061118, -0.45990349,  0.32999083, -0.19347339,  0.05308359,
         0.08836869, -0.22805226,  0.36317137, -0.49102159,

In [6]:
df=pd.DataFrame(data)
df.head(5)

,X,Y
0,60,-0.304811
1,63,0.167356
2,66,-0.026551
3,69,-0.114785
4,72,0.253823


In [7]:
#Seperating Input and Output Features
Y=df.iloc[:,1]
X=df.iloc[:,0]

In [8]:
#Scaling input features
from cuml.preprocessing import StandardScaler
ss=StandardScaler()
X_scaled=ss.fit_transform(pd.DataFrame(X))

In [9]:
X_scaled=pd.DataFrame(X_scaled)
#Adding a column of Ones
X_scaled.insert(loc=0,value=1.0,name='Ones')
X_scaled

,Ones,0
0,1.0,-1.714816
1,1.0,-1.680173
2,1.0,-1.645531
3,1.0,-1.610888
4,1.0,-1.576245
...,...,...
95,1.0,1.576245
96,1.0,1.610888
97,1.0,1.645531
98,1.0,1.680173


In [10]:
#Y=pd.DataFrame(Y)

In [11]:
#train-test_split
from cuml.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_scaled,Y,test_size=0.30,random_state=42)

In [12]:
#finding beta's using LSE
A=X_train.values.T.dot(X_train.values)
B=np.linalg.inv(A)
C=B.dot(X_train.values.T)
beta=C.dot(Y_train.values)

In [13]:
beta

array([-0.01032002, -0.06573132])

In [14]:
#Predicting and Testing
Y_predict=X_test.values.dot(beta)
from cuml import metrics
metrics.r2_score(Y_test,Y_predict)

-0.05702010097418708

In [15]:
#The fit model is underfit as it shows a low R2 score. So fiiting a higher order polynomial function of order 5

In [16]:
#adding powers of X
from cuml.preprocessing import PolynomialFeatures
pf=PolynomialFeatures(50)
X50=pf.fit_transform(pd.DataFrame(X))
X50

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,1.0,60.0,3600.0,216000.0,1.296000e+07,7.776000e+08,4.665600e+10,2.799360e+12,1.679616e+14,1.007770e+16,...,8.020497e+72,4.812298e+74,2.887379e+76,1.732427e+78,1.039456e+80,6.236738e+81,3.742043e+83,2.245226e+85,1.347135e+87,8.082813e+88
1,1.0,63.0,3969.0,250047.0,1.575296e+07,9.924365e+08,6.252350e+10,3.938981e+12,2.481558e+14,1.563381e+16,...,5.928742e+73,3.735107e+75,2.353118e+77,1.482464e+79,9.339524e+80,5.883900e+82,3.706857e+84,2.335320e+86,1.471252e+88,9.268885e+89
2,1.0,66.0,4356.0,287496.0,1.897474e+07,1.252333e+09,8.265395e+10,5.455161e+12,3.600406e+14,2.376268e+16,...,3.993019e+74,2.635392e+76,1.739359e+78,1.147977e+80,7.576648e+81,5.000588e+83,3.300388e+85,2.178256e+87,1.437649e+89,9.488483e+90
3,1.0,69.0,4761.0,328509.0,2.266712e+07,1.564031e+09,1.079182e+11,7.446353e+12,5.137984e+14,3.545209e+16,...,2.470658e+75,1.704754e+77,1.176281e+79,8.116336e+80,5.600272e+82,3.864187e+84,2.666289e+86,1.839740e+88,1.269420e+90,8.759000e+91
4,1.0,72.0,5184.0,373248.0,2.687386e+07,1.934918e+09,1.393141e+11,1.003061e+13,7.222041e+14,5.199870e+16,...,1.414596e+76,1.018509e+78,7.333264e+79,5.279950e+81,3.801564e+83,2.737126e+85,1.970731e+87,1.418926e+89,1.021627e+91,7.355714e+92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,345.0,119025.0,41063625.0,1.416695e+10,4.887598e+12,1.686221e+15,5.817463e+17,2.007025e+20,6.924236e+22,...,1.123525e+104,3.876163e+106,1.337276e+109,4.613603e+111,1.591693e+114,5.491340e+116,1.894512e+119,6.536068e+121,2.254943e+124,7.779555e+126
96,1.0,348.0,121104.0,42144192.0,1.466618e+10,5.103830e+12,1.776133e+15,6.180943e+17,2.150968e+20,7.485369e+22,...,1.602319e+104,5.576070e+106,1.940472e+109,6.752844e+111,2.349990e+114,8.177965e+116,2.845932e+119,9.903842e+121,3.446537e+124,1.199395e+127
97,1.0,351.0,123201.0,43243551.0,1.517849e+10,5.327649e+12,1.870005e+15,6.563717e+17,2.303864e+20,8.086564e+22,...,2.278200e+104,7.996481e+106,2.806765e+109,9.851744e+111,3.457962e+114,1.213745e+117,4.260244e+119,1.495346e+122,5.248663e+124,1.842281e+127
98,1.0,354.0,125316.0,44361864.0,1.570410e+10,5.559251e+12,1.967975e+15,6.966631e+17,2.466188e+20,8.730304e+22,...,3.229489e+104,1.143239e+107,4.047066e+109,1.432661e+112,5.071621e+114,1.795354e+117,6.355553e+119,2.249866e+122,7.964524e+124,2.819442e+127


In [17]:
#scaling features
ss1=StandardScaler()
X5_scaled=ss1.fit_transform(X50.iloc[:,1:])

In [18]:
#Inserting columns of One
X5_scaled.insert(loc=0,value=1.0,name='Ones')

In [19]:
#Train -Test Split
from cuml.model_selection import train_test_split
X_train1,X_test1,Y_train1,Y_test1=train_test_split(X5_scaled,Y,test_size=0.30,random_state=42)

In [ ]:
#Finding beta's for 5th order hypothesis function
A=X_train1.values.T.dot(X_train1.values)
B=np.linalg.inv(A)
C=B.dot(X_train1.values.T)
beta1=C.dot(Y_train1.values)

In [23]:
beta1

array([-1.09163661e-04,  7.65086692e+03, -8.79759683e+04,  4.56484451e+05,
       -1.28807305e+06,  1.88677926e+06, -8.47938323e+05, -9.58963157e+05,
        5.94000974e+05,  1.10832745e+05,  2.33057486e+06, -3.05788324e+06,
        4.70315431e+05, -5.45570070e+05, -6.00244366e+05,  2.72167985e+06,
        1.06941034e+06, -1.15913115e+06, -3.01264702e+06,  2.72055733e+06,
       -2.83294943e+06,  1.56683690e+06, -3.76352399e+05,  9.01184183e+05,
        2.71004162e+06, -2.36346839e+06,  1.63343793e+05, -1.03076621e+06,
       -5.51140951e+05, -7.85754618e+05,  2.39726958e+06, -1.20628969e+05,
       -1.14789132e+05,  4.53098783e+05, -6.71980429e+05, -1.21954472e+06,
        4.51164144e+05,  3.62400789e+05,  1.33322451e+05,  1.48906154e+06,
       -9.55728712e+05, -3.54883915e+05, -1.45879241e+06,  2.38136437e+06,
       -1.66917058e+06,  1.40815595e+06, -4.18515148e+05, -1.36528706e+06,
        9.93726423e+05,  5.01022140e+05, -4.32099865e+05])

In [24]:
#prediciting and testing
Y_predict1=X_test1.values.dot(beta1)
from cuml import metrics
metrics.r2_score(Y_test1,Y_predict1)

-0.1637105093206328

In [27]:
#finding beta's with ridge regression
lamda=1000
A=X_train1.values.T.dot(X_train1.values)
I=np.identity(A.shape[0])
A1=np.add(A,lamda*I)
B=np.linalg.inv(A1)
C=B.dot(X_train1.values.T)
beta2=C.dot(Y_train1.values)
beta2

array([-8.31021338e-04,  1.52279983e-03,  1.61683979e-03,  1.62084273e-03,
        1.54492908e-03,  1.41783079e-03,  1.26296236e-03,  1.09553895e-03,
        9.25100556e-04,  7.57865553e-04,  5.98078828e-04,  4.48676748e-04,
        3.11621565e-04,  1.88103174e-04,  7.86927082e-05, -1.65242596e-05,
       -9.78294939e-05, -1.65769724e-04, -2.21071253e-04, -2.64571397e-04,
       -2.97165392e-04, -3.19766697e-04, -3.33278397e-04, -3.38573439e-04,
       -3.36481733e-04, -3.27782395e-04, -3.13199786e-04, -2.93402249e-04,
       -2.69002728e-04, -2.40560629e-04, -2.08584481e-04, -1.73535039e-04,
       -1.35828623e-04, -9.58404965e-05, -5.39082021e-05, -1.03347635e-05,
        3.46082778e-05,  8.06780210e-05,  1.27657567e-04,  1.75353583e-04,
        2.23594069e-04,  2.72226308e-04,  3.21115006e-04,  3.70140586e-04,
        4.19197646e-04,  4.68193559e-04,  5.17047197e-04,  5.65687779e-04,
        6.14053829e-04,  6.62092228e-04,  7.09757358e-04])

In [28]:
#Predicting and Testing
Y_predict=X_test1.values.dot(beta2)
from cuml import metrics
metrics.r2_score(Y_test1,Y_predict)

-0.0025586042110545826

In [ ]:
#Question 2 starts from here

In [1]:
import pandas

In [25]:
!unzip "archive.zip"

Archive:  archive.zip
  inflating: IMDB Dataset.csv        


In [3]:
#Laod first 1ooo rows of IMDB Dataset
df=pd.read_csv('IMDB_Dataset.csv',nrows=1000)

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
#Create a list of list named Corpus whose each list is a review (document) from IMDB dataset
corpus=[]
for i in range(len(df)):
    corpus.append(df['review'][i])

In [6]:
corpus[10]

'Phil the Alien is one of those quirky films where the humour is based around the oddness of everything rather than actual punchlines.<br /><br />At first it was very odd and pretty funny but as the movie progressed I didn\'t find the jokes or oddness funny anymore.<br /><br />Its a low budget film (thats never a problem in itself), there were some pretty interesting characters, but eventually I just lost interest.<br /><br />I imagine this film would appeal to a stoner who is currently partaking.<br /><br />For something similar but better try "Brother from another planet"'

In [7]:
# remove non-aplha characters and converting words to lower case in each document
corpus_final=[]
for i in range(len(corpus)):
    corpus_final.append(" ".join([word.lower() for word in corpus[i].split() if word.isalpha()]))

In [8]:
corpus_final[10]

'phil the alien is one of those quirky films where the humour is based around the oddness of everything rather than actual first it was very odd and pretty funny but as the movie progressed i find the jokes or oddness funny a low budget film never a problem in there were some pretty interesting but eventually i just lost imagine this film would appeal to a stoner who is currently something similar but better try from another'

In [9]:
#creating binary bow vector as dictionary of dictionary
#Each external dictionary will have key as document number and internal dictionaries have the keys as the word present in the document
document_frequency={}
for i in range(len(corpus_final)):
    word_frequency={}
    for word in corpus_final[i].split():
        word_frequency[word]=1
    document_frequency[i]=word_frequency

In [10]:
document_frequency[0]

{'one': 1,
 'of': 1,
 'the': 1,
 'other': 1,
 'reviewers': 1,
 'has': 1,
 'mentioned': 1,
 'that': 1,
 'after': 1,
 'watching': 1,
 'just': 1,
 'oz': 1,
 'episode': 1,
 'be': 1,
 'they': 1,
 'are': 1,
 'as': 1,
 'this': 1,
 'is': 1,
 'exactly': 1,
 'what': 1,
 'happened': 1,
 'with': 1,
 'first': 1,
 'thing': 1,
 'struck': 1,
 'me': 1,
 'about': 1,
 'was': 1,
 'its': 1,
 'brutality': 1,
 'and': 1,
 'unflinching': 1,
 'scenes': 1,
 'which': 1,
 'set': 1,
 'in': 1,
 'right': 1,
 'from': 1,
 'word': 1,
 'trust': 1,
 'not': 1,
 'a': 1,
 'show': 1,
 'for': 1,
 'faint': 1,
 'hearted': 1,
 'or': 1,
 'pulls': 1,
 'no': 1,
 'punches': 1,
 'regards': 1,
 'to': 1,
 'sex': 1,
 'classic': 1,
 'use': 1,
 'called': 1,
 'nickname': 1,
 'given': 1,
 'oswald': 1,
 'maximum': 1,
 'security': 1,
 'state': 1,
 'it': 1,
 'focuses': 1,
 'mainly': 1,
 'on': 1,
 'emerald': 1,
 'an': 1,
 'experimental': 1,
 'section': 1,
 'prison': 1,
 'where': 1,
 'all': 1,
 'cells': 1,
 'have': 1,
 'glass': 1,
 'fronts': 1,
 

In [11]:
#Converting dicitonaries of dicitonaries into DataFrame; setting NaN as 0 (for terms which are not present in particular docme)
bow1=pandas.DataFrame(document_frequency)
bow1.fillna(0,inplace=True)
bow1.shape

(14719, 1000)

In [12]:
bow= pd.DataFrame.from_pandas(bow1)

In [13]:
bow

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
one,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
of,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
the,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
other,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
reviewers,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
offices,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
intuition,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
stubborn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
masking,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
sum_column = bow.sum(axis=1)
sum_column


one          501.0
of           954.0
the          994.0
other        242.0
reviewers      7.0
             ...  
offices        1.0
intuition      1.0
stubborn       1.0
masking        1.0
planes         1.0
Length: 14719, dtype: float64

In [16]:
Val=sum_column.values

In [19]:
Val=np.asarray(Val)

In [17]:
type(sum_column)

cupy._core.core.ndarray

In [21]:
A=bow1.values
B=A.T

In [22]:
cooccurrence_matrix=A.dot(B)

In [23]:
cooccurrence_matrix=np.asarray(cooccurrence_matrix)

In [24]:
type(cooccurrence_matrix)

cupy._core.core.ndarray

In [25]:
np.fill_diagonal(cooccurrence_matrix,0)

In [26]:
cooccurrence_matrix.shape

(14719, 14719)

In [27]:
cooccurrence_matrix

array([[  0., 496., 500., ...,   1.,   1.,   1.],
       [496.,   0., 950., ...,   1.,   1.,   1.],
       [500., 950.,   0., ...,   1.,   1.,   1.],
       ...,
       [  1.,   1.,   1., ...,   0.,   1.,   1.],
       [  1.,   1.,   1., ...,   1.,   0.,   1.],
       [  1.,   1.,   1., ...,   1.,   1.,   0.]])

In [ ]:
print("see")

In [55]:
PPMI=[]
for i in range(500):
    new=[]
    for j in range(500):
        if cooccurrence_matrix[i][j]<=0 :
            new.append(0.0)
        else:
            new.append( math.log((cooccurrence_matrix[i][j]*1000)/(Val[i]*Val[j])) )
    print("*",end=" ")
    PPMI.append(new)

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

In [56]:
PPMI=np.asarray(PPMI)
PPMI.shape

(500, 500)

In [57]:
PPMI

array([[ 0.        ,  0.03706143,  0.00402007, ...,  0.14110284,
         0.41828219,  0.24073268],
       [ 0.03706143,  0.        ,  0.00181639, ...,  0.04709161,
         0.04709161,  0.03604177],
       [ 0.00402007,  0.00181639,  0.        , ...,  0.00601807,
         0.00601807,  0.00601807],
       ...,
       [ 0.14110284,  0.04709161,  0.00601807, ...,  0.        ,
        -1.24818106,  0.5250936 ],
       [ 0.41828219,  0.04709161,  0.00601807, ..., -1.24818106,
         0.        ,  0.44349495],
       [ 0.24073268,  0.03604177,  0.00601807, ...,  0.5250936 ,
         0.44349495,  0.        ]])